## Basic Patterns


### Assertions


In [ ]:
def mean(xs):
    assert len(xs) > 0, "mean() requires non-empty list"
    return sum(xs) / len(xs)

print(mean([1,2,3]))
# mean([])  # would raise AssertionError

### Testing Hooks


In [ ]:
def is_pal(s: str) -> bool:
    """
    Return True if s is a palindrome.

    Examples:
    >>> is_pal('racecar')
    True
    >>> is_pal('abc')
    False
    """
    return s == s[::-1]

assert is_pal("madam") is True
assert is_pal("nope") is False
# To run doctests in a script:
# if __name__ == "__main__":
#     import doctest; doctest.testmod()

### Unit Testing


In [ ]:
# Pytest discovers functions named test_* in files/test modules.
# Example tests (showing the style—run with `pytest -q` in a terminal):

def add(a, b): return a + b

def test_add_basic():
    assert add(2, 3) == 5

## Parametrized Tests


In [ ]:
from datetime import datetime, timedelta

import pytest

@pytest.mark.parametrize(
    "a,b,expected",
    [
        pytest.param(
            datetime(2001, 12, 12), datetime(2001, 12, 11), timedelta(1), id="forward"
        ),
        pytest.param(
            datetime(2001, 12, 11), datetime(2001, 12, 12), timedelta(-1), id="backward"
        ),
    ],
)
def test_timedistance_v3(a, b, expected):
    diff = a - b
    assert diff == expected

## Markers


- Markers allow you to add metadata to your tests, enabling selective test execution and organization.

### Built-in Markers

In [ ]:
import pytest
import sys

# Skip a test
@pytest.mark.skip(reason="Not implemented yet")
def test_feature_a():
    assert False

# Skip conditionally
@pytest.mark.skipif(sys.version_info < (3, 10), reason="Requires Python 3.10+")
def test_feature_b():
    assert True

# Mark test as expected to fail
@pytest.mark.xfail(reason="Known bug #123")
def test_buggy_feature():
    assert 1 / 0

### Custom Markers

In [ ]:
import pytest

# Mark tests by category
@pytest.mark.slow
def test_database_query():
    # Long-running test
    assert True

@pytest.mark.integration
def test_api_endpoint():
    assert True

@pytest.mark.unit
def test_add():
    assert 1 + 1 == 2

### Running Tests by Marker

In [ ]:
# Run only slow tests
pytest -m slow

# Run everything except slow tests
pytest -m "not slow"

# Combine markers
pytest -m "integration and not slow"

## Fixtures


- provides a defined, reliable and consistent context for the tests. This could include environment (for example a database configured with known parameters) or content (such as a dataset).


In [ ]:
import pytest


class Fruit:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return self.name == other.name


@pytest.fixture
def my_fruit():
    return Fruit("apple")


@pytest.fixture
def fruit_basket(my_fruit):
    return [Fruit("banana"), my_fruit]


def test_my_fruit_in_basket(my_fruit, fruit_basket):
    assert my_fruit in fruit_basket

### Fixture Scopes

- Control how long a fixture lives and how often it's recreated:

In [ ]:
import pytest

# Function scope (default): created for each test
@pytest.fixture(scope="function")
def func_fixture():
    return "new for each test"

# Class scope: created once per test class
@pytest.fixture(scope="class")
def class_fixture():
    return "shared across class"

# Module scope: created once per module
@pytest.fixture(scope="module")
def module_fixture():
    return "shared across module"

# Session scope: created once per test session
@pytest.fixture(scope="session")
def session_fixture():
    return "shared across all tests"

### Setup and Teardown

- Use `yield` for cleanup after tests:

In [ ]:
import pytest
import os

@pytest.fixture
def temp_file():
    # Setup
    file = open("test.txt", "w")
    file.write("test data")
    file.close()
    
    # Provide fixture value
    yield "test.txt"
    
    # Teardown (runs after test completes)
    import os
    os.remove("test.txt")

def test_file_exists(temp_file):
    assert os.path.exists(temp_file)

### Autouse Fixtures

- Automatically run for every test without explicit request:

In [ ]:
import pytest

@pytest.fixture(autouse=True)
def reset_state():
    # Runs before every test automatically
    global counter
    counter = 0

### Fixture Factories

- Return a function to create multiple instances:

In [ ]:
import pytest

@pytest.fixture
def make_user():
    def _make_user(name, age):
        return {"name": name, "age": age}
    return _make_user

def test_users(make_user):
    user1 = make_user("Alice", 30)
    user2 = make_user("Bob", 25)
    assert user1["age"] > user2["age"]

### Parametrized Fixtures

- Create multiple fixture variations

In [ ]:
import pytest

@pytest.fixture(params=[1, 2, 3])
def number(request):
    return request.param

def test_number(number):
    # This test runs 3 times with number=1, 2, 3
    assert number > 0

### Built-in Fixtures

- Pytest provides useful built-in fixtures:

In [ ]:
def test_tmp_path(tmp_path):
    # tmp_path: pathlib.Path to temp directory
    file = tmp_path / "test.txt"
    file.write_text("content")
    assert file.read_text() == "content"

def test_capsys(capsys):
    # capsys: capture stdout/stderr
    print("hello")
    captured = capsys.readouterr()
    assert captured.out == "hello\n"

def test_monkeypatch(monkeypatch):
    # monkeypatch: modify objects/environment
    monkeypatch.setenv("USER", "test_user")
    import os
    assert os.getenv("USER") == "test_user"

## Hooks


- 

In [ ]:
# conftest.py
import pytest

def pytest_configure(config):
    """Called after command line options are parsed."""
    config.addinivalue_line("markers", "smoke: mark test as smoke test")

def pytest_collection_modifyitems(config, items):
    """Modify collected test items."""
    for item in items:
        if "slow" in item.keywords:
            item.add_marker(pytest.mark.slow)

def pytest_runtest_setup(item):
    """Called before running each test."""
    print(f"\nSetting up {item.name}")

def pytest_runtest_teardown(item):
    """Called after running each test."""
    print(f"\nTearing down {item.name}")

## Coverage & CLI


In [ ]:
# From terminal:
#   pytest -q
#   pytest -q -k "keyword"          # subset by name
#   pytest -q -x                     # stop after first failure
#   pytest --maxfail=1 --disable-warnings -q
#   coverage run -m pytest && coverage html
print("Run pytest/coverage from terminal; see comments for common commands.")